# **TEAM 74: STOCK PRICE FORECASTING**

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import datetime
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss
from scipy import stats
from fredapi import Fred

# ============================================
# Список тикеров (акции, товары, индексы, облигации, валюты)
# ============================================

In [2]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# Список тикеров (акции, товары, индексы, облигации, валюты)
most_traded_tickers = [
    # Акции
    "AAPL", "MSFT", "NVDA", "TSLA", "AMZN", "GOOG", "META", "JPM", "WMT", "NFLX",
    "BABA", "DIS", "PFE", "VZ", "KO", "INTC", "CSCO", "ADBE", "CMCSA", "T",

    # Сырьевые товары
    "CL=F", "GC=F", "SI=F", "HG=F", "ZS=F",

    # Индексы
    "^GSPC", "^DJI", "^IXIC", "^RUT", "^VIX",

    # Облигации
    "^TNX", "^IRX", "^TYX",

    # Валюты
    "EURUSD=X", "JPYUSD=X", "GBPUSD=X", "AUDUSD=X", "CADUSD=X",
    "CHFUSD=X", "CNYUSD=X", "SGDUSD=X", "HKDUSD=X"
]

# Группы активов для анализа
asset_classes = {
    "Equities": [
        "AAPL", "MSFT", "NVDA", "TSLA", "AMZN", "GOOG", "META", "JPM", "WMT", "NFLX",
        "BABA", "DIS", "PFE", "VZ", "KO", "INTC", "CSCO", "ADBE", "CMCSA", "T"
    ],
    "Commodities": ["CL=F", "GC=F", "SI=F", "HG=F", "ZS=F"],
    "Indices": ["^GSPC", "^DJI", "^IXIC", "^RUT", "^VIX"],
    "Bond Indices": ["^TNX", "^IRX", "^TYX"],
    "Currencies": [
        "EURUSD=X", "JPYUSD=X", "GBPUSD=X", "AUDUSD=X", "CADUSD=X",
        "CHFUSD=X", "CNYUSD=X", "SGDUSD=X", "HKDUSD=X"
    ]
}




# ============================================
# 0. Загрузка данных
# ============================================

In [ ]:
base_dict = {}
data_list = []

for ticker in tqdm(most_traded_tickers):
    # Загрузка данных и названия компании
    data = yf.download(ticker, start='1990-01-01', interval='1d')
    info = yf.Ticker(ticker).info
    company_name = info.get("shortName", ticker)  # Используем тикер, если название недоступно

    # Добавление названия компании и тикера в данные
    data['Ticker'] = ticker
    data['Company Name'] = company_name
    data_list.append(data)

# Объединение данных в один DataFrame
full_data = pd.concat(data_list)
full_data.reset_index(inplace=True)  # Сброс индекса для красоты

# Сохранение в красивую таблицу
full_data = full_data[['Date', 'Ticker', 'Company Name', 'Open', 'High', 'Low', 'Close', 'Volume']]
full_data.to_csv("data_with_company_names.csv", index=False)

# ============================================
# 1. Изучение структуры данных
# ============================================

In [ ]:
import pandas as pd
import yfinance as yf
from tabulate import tabulate
from tqdm import tqdm

# Словарь для хранения полных названий компаний по тикерам
full_names = {
    "AAPL": "Apple Inc.", "MSFT": "Microsoft Corporation", "NVDA": "NVIDIA Corporation", "TSLA": "Tesla Inc.",
    "AMZN": "Amazon.com Inc.", "GOOG": "Alphabet Inc.", "META": "Meta Platforms, Inc.", "JPM": "JPMorgan Chase & Co.",
    "WMT": "Walmart Inc.", "NFLX": "Netflix, Inc.", "BABA": "Alibaba Group Holding Limited",
    "DIS": "The Walt Disney Company", "PFE": "Pfizer Inc.", "VZ": "Verizon Communications Inc.",
    "KO": "The Coca-Cola Company", "INTC": "Intel Corporation", "CSCO": "Cisco Systems, Inc.",
    "ADBE": "Adobe Inc.", "CMCSA": "Comcast Corporation", "T": "AT&T Inc.",
    "CL=F": "Crude Oil Futures", "GC=F": "Gold Futures", "SI=F": "Silver Futures", 
    "HG=F": "Copper Futures", "ZS=F": "Soybean Futures",
    "^GSPC": "S&P 500", "^DJI": "Dow Jones Industrial Average", "^IXIC": "NASDAQ Composite", 
    "^RUT": "Russell 2000", "^VIX": "Volatility Index",
    "^TNX": "10-Year Treasury Yield", "^IRX": "13-Week Treasury Bill Yield", "^TYX": "30-Year Treasury Yield",
    "EURUSD=X": "Euro/US Dollar", "JPYUSD=X": "Japanese Yen/US Dollar", "GBPUSD=X": "British Pound/US Dollar",
    "AUDUSD=X": "Australian Dollar/US Dollar", "CADUSD=X": "Canadian Dollar/US Dollar",
    "CHFUSD=X": "Swiss Franc/US Dollar", "CNYUSD=X": "Chinese Yuan/US Dollar", 
    "SGDUSD=X": "Singapore Dollar/US Dollar", "HKDUSD=X": "Hong Kong Dollar/US Dollar"
}

# Загружаем данные для каждого тикера и добавляем их в словарь
base_dict = {}
for ticker in tqdm(full_names.keys()):
    base_dict[ticker] = yf.download(ticker, start='1990-01-01', interval='1d')

# ============================================
# A) Проверка на наличие необходимых колонок и структура данных
# ============================================

def check_columns_and_structure(data, ticker):
    # Вывод первых 5 строк с названием компании и тикером
    table = data[["Open", "High", "Low", "Close", "Volume"]].head(5)
    table["Ticker"] = ticker
    table["Full Name"] = full_names.get(ticker, "Unknown")
    print(tabulate(table, headers="keys", tablefmt="grid", showindex=False))
    return table

# Проверка колонок и структуры данных для всех активов
column_structure_data = {ticker: check_columns_and_structure(data, ticker) for ticker, data in base_dict.items()}

# ============================================
# B) Проверка структуры типов данных
# ============================================

def check_data_types(data, ticker):
    # Структура типов данных, первые 5 строк
    types_table = pd.DataFrame(data.dtypes, columns=["Data Type"]).reset_index()
    types_table["Ticker"] = ticker
    types_table["Full Name"] = full_names.get(ticker, "Unknown")
    print(tabulate(types_table, headers="keys", tablefmt="grid", showindex=False))
    return types_table

# Проверка структуры типов данных для всех активов
data_types_data = {ticker: check_data_types(data, ticker) for ticker, data in base_dict.items()}

# ============================================
# C) Проверка частоты и пропусков
# ============================================

def check_frequency_and_missing(data, ticker):
    freq = pd.infer_freq(data.index)
    missing_data = data.isna().sum().reset_index()
    missing_data.columns = ["Column", "Missing Values"]
    missing_data["Ticker"] = ticker
    missing_data["Full Name"] = full_names.get(ticker, "Unknown")
    print(f"\nЧастота данных для {ticker} ({full_names.get(ticker)}): {freq}")
    print(tabulate(missing_data, headers="keys", tablefmt="grid", showindex=False))
    return freq, missing_data

# Проверка частоты и пропусков для всех активов
frequency_missing_data = {ticker: check_frequency_and_missing(data, ticker) for ticker, data in base_dict.items()}

# ============================================
# D) Разделение на группы активов и корректное определение
# ============================================

asset_classes = {
    "Equities": ["AAPL", "MSFT", "NVDA", "TSLA", "AMZN", "GOOG", "META", "JPM", "WMT", "NFLX",
                 "BABA", "DIS", "PFE", "VZ", "KO", "INTC", "CSCO", "ADBE", "CMCSA", "T"],
    "Commodities": ["CL=F", "GC=F", "SI=F", "HG=F", "ZS=F"],
    "Indices": ["^GSPC", "^DJI", "^IXIC", "^RUT", "^VIX"],
    "Bond Indices": ["^TNX", "^IRX", "^TYX"],
    "Currencies": ["EURUSD=X", "JPYUSD=X", "GBPUSD=X", "AUDUSD=X", "CADUSD=X",
                   "CHFUSD=X", "CNYUSD=X", "SGDUSD=X", "HKDUSD=X"]
}

def check_asset_groups():
    group_data = []
    for group, tickers in asset_classes.items():
        for ticker in tickers:
            group_data.append({
                "Asset Group": group,
                "Ticker": ticker,
                "Full Name": full_names.get(ticker, "Unknown")
            })
    
    group_df = pd.DataFrame(group_data)
    print(tabulate(group_df, headers="keys", tablefmt="grid", showindex=False))
    return group_df

# Разделение на группы активов
asset_groups_data = check_asset_groups()


# ============================================
# 2. Очистка выбросов по всем    активам
# ============================================

In [ ]:

import pandas as pd

from IPython.display import display

# Описательная статистика по каждому активу в словаре base_dict

for ticker, df in base_dict.items():

    print(f"\nОписательная статистика для {ticker}:")

    display(df.describe())

# Функция для удаления экстремальных выбросов по строгим критериям (например, 0.1% и 99.9% перцентили)

def remove_extreme_outliers(data, column, ticker):

    lower_threshold = data[column].quantile(0.001)  # 0.1-й перцентиль

    upper_threshold = data[column].quantile(0.999)  # 99.9-й перцентиль



    # Отбираем выбросы и очищенные данные

    outliers = data[(data[column] < lower_threshold) | (data[column] > upper_threshold)]

    cleaned_data = data[(data[column] >= lower_threshold) & (data[column] <= upper_threshold)]

    

    return cleaned_data, outliers



# Словари для хранения очищенных данных и выбросов

cleaned_data_dict = {}

outliers_data_dict = {}



# Очистка выбросов для всех активов в словаре

for ticker, df in base_dict.items():

    if isinstance(df, pd.DataFrame) and 'Adj Close' in df.columns:

        cleaned_data, outliers = remove_extreme_outliers(df, 'Adj Close', ticker)

        

        cleaned_data_dict[ticker] = cleaned_data

        outliers_data_dict[ticker] = outliers

    else:

        print(f"Пропуск {ticker}: неверный формат данных или отсутствует столбец 'Adj Close'.")



# Функция для форматирования таблицы с первыми 10 значениями

def display_summary_table(data, title, info):

    summary_table = data.head(10)  # только первые 10 строк

    styled_table = summary_table.style.set_caption(f"{title} ({info})").set_properties(**{

        'border': '1px solid black',

        'padding': '8px'

    }).set_table_styles([

        dict(selector="thead", props=[("background-color", "#CCCCCC"), ("color", "black")]),

        dict(selector="tbody tr:nth-child(even)", props=[("background-color", "#F2F2F2")]),

        dict(selector="tbody tr:nth-child(odd)", props=[("background-color", "white")])

    ])

    display(styled_table)



# Вывод информации по каждому активу

for ticker, outliers in outliers_data_dict.items():

    num_outliers = len(outliers)

    num_total = len(base_dict[ticker]) if ticker in base_dict else 0

    num_cleaned = num_total - num_outliers

    

    # Отображение информации о выбросах

    if not outliers.empty:

        display_summary_table(outliers, f"Выбросы для {ticker}", f"Найдено выбросов: {num_outliers}")

    

    # Отображение информации об очищенных данных

    display_summary_table(cleaned_data_dict[ticker], f"Очищенные данные для {ticker}", f"Осталось после удаления: {num_cleaned}")

# ============================================
# 3. Визуализация трендов
# ============================================

In [ ]:
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose




# Функция для визуализации трендов, средних и стандартного отклонения

def visualize_trends_and_stats(data, ticker, title, start_date, end_date):

    plt.figure(figsize=(10, 5))

    plt.plot(data.index, data['Adj Close'], label='Price', color='blue')

    

    rolling_mean = data['Adj Close'].rolling(window=365).mean()

    rolling_std = data['Adj Close'].rolling(window=365).std()



    plt.plot(data.index, rolling_mean, label='365-Day Rolling Mean', color='green')

    plt.plot(data.index, rolling_std, label='365-Day Rolling Std', color='red')

    

    plt.title(f'{title} ({start_date.date()} - {end_date.date()})')

    plt.xlabel('Date')

    plt.ylabel('Adjusted Closing Price')

    plt.legend()

    plt.grid(True)

    plt.tight_layout()

    plt.show()



# Функция для мультипликативной декомпозиции временного ряда

def plot_multiplicative_decomposition(data, ticker, title):

    result = seasonal_decompose(data['Adj Close'], model='multiplicative', period=365)

    fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 8), sharex=True)

    

    result.observed.plot(ax=ax1, color='blue')

    ax1.set_ylabel('Observed')

    result.trend.plot(ax=ax2, color='blue')

    ax2.set_ylabel('Trend')

    result.seasonal.plot(ax=ax3, color='blue')

    ax3.set_ylabel('Seasonal')

    result.resid.plot(ax=ax4, color='blue')

    ax4.set_ylabel('Residual')

    

    plt.suptitle(f'Multiplicative Decomposition for {title}', y=1.02)

    plt.tight_layout()

    plt.show()



# Визуализация данных для каждого актива

for ticker, data in cleaned_data_dict.items():

    start_date = data.index.min()

    end_date = data.index.max()

    title = f"{full_names.get(ticker, ticker)}"

    

    # Визуализация трендов, средних и стандартного отклонения

    visualize_trends_and_stats(data, ticker, title, start_date, end_date)

    

    # Мультипликативная декомпозиция

    plot_multiplicative_decomposition(data, ticker, title)

# Prophet forecasting

In [ ]:
from prophet import Prophet

import pandas as pd

import matplotlib.pyplot as plt



# Задаем параметры модели Prophet и добавляем пользовательскую сезонность

def create_and_train_prophet_model(data):

    model = Prophet(seasonality_mode='additive', yearly_seasonality=True)

    model.add_seasonality(name='custom_yearly', period=30.5, fourier_order=5)

    model.fit(data)

    return model



# Функция для построения прогноза и визуализации

def plot_prophet_forecast(data, ticker, title, forecast_period=365):

    # Разделяем данные на обучающую и тестовую выборки

    train_data = data[:-forecast_period]

    

    # Подготовка данных для Prophet

    prophet_data = train_data.reset_index().rename(columns={'Date': 'ds', 'Adj Close': 'y'})

    

    # Создание и обучение модели

    model = create_and_train_prophet_model(prophet_data)

    

    # Создание датафрейма для прогноза на 365 дней вперед

    future = model.make_future_dataframe(periods=forecast_period)

    forecast = model.predict(future)

    

    # Визуализация прогноза

    fig = model.plot(forecast)

    plt.title(f'{title} - Prophet Forecast')

    plt.xlabel('Date')

    plt.ylabel('Price')

    plt.grid(True)

    plt.show()



# Визуализация прогноза для каждого актива

for ticker, data in cleaned_data_dict.items():

    title = f"{full_names.get(ticker, ticker)}"

    plot_prophet_forecast(data, ticker, title)

# ============================================
# 4. Групповая визуализация для каждой группы активов
# ============================================

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd


# Функция для групповой визуализации

def plot_grouped_assets(data_dict, groups, cleaned_data_dict):

    for group_name, tickers in groups.items():

        plt.figure(figsize=(12, 8))

        start_date, end_date = None, None  # Переменные для хранения периода



        for ticker in tickers:

            data = cleaned_data_dict.get(ticker)

            if data is not None:

                plt.plot(data.index, data['Adj Close'], label=f"{full_names.get(ticker, ticker)} ({ticker})")

                

                # Обновляем значения начала и конца периода

                if start_date is None or data.index.min() < start_date:

                    start_date = data.index.min()

                if end_date is None or data.index.max() > end_date:

                    end_date = data.index.max()



        plt.title(f"{group_name} (Period: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')})")

        plt.xlabel("Date")

        plt.ylabel("Adjusted Closing Price")

        plt.legend(loc="best")

        plt.grid(True)

        plt.show()



# Вызов функции для групповой визуализации

plot_grouped_assets(base_dict, asset_classes, cleaned_data_dict)

# ============================================
# 5. Проверка и подсчет пропусков в данных
# ============================================

In [ ]:
import pandas as pd

from IPython.display import display



# Функция для проверки пропусков в данных

def check_missing_data(cleaned_data_dict):

    # Создаем список для сбора информации о пропусках

    missing_data_summary = []



    # Проходим по каждому активу в очищенных данных

    for ticker, data in cleaned_data_dict.items():

        if data is not None:

            missing_count = data.isnull().sum().sum()  # Количество пропусков

            total_count = data.shape[0] * data.shape[1]  # Общее количество элементов

            missing_percentage = (missing_count / total_count) * 100  # Процент пропусков

            

            # Добавляем данные в таблицу

            missing_data_summary.append({

                'Ticker': ticker,

                'Full Name': full_names.get(ticker, ticker),

                'Total Missing Values': missing_count,

                'Missing Percentage (%)': round(missing_percentage, 2)

            })



    # Создаем DataFrame для отображения результатов

    missing_data_df = pd.DataFrame(missing_data_summary)

    display(missing_data_df)



# Вызов функции для проверки пропусков в очищенных данных

check_missing_data(cleaned_data_dict)

# Функция для заполнения пропусков линейной интерполяцией

In [ ]:
import pandas as pd

from IPython.display import display



# Функция для заполнения пропусков линейной интерполяцией

def fill_missing_data(cleaned_data_dict):

    filled_data_dict = {}

    for ticker, data in cleaned_data_dict.items():

        if data is not None:

            # Заполняем пропуски методом линейной интерполяции

            filled_data = data.interpolate(method='linear')

            # Заполнение оставшихся (если есть в начале или конце) методом ближайшего соседа

            filled_data = filled_data.fillna(method='bfill').fillna(method='ffill')

            filled_data_dict[ticker] = filled_data

    return filled_data_dict



# Заполнение пропусков в данных

filled_data_dict = fill_missing_data(cleaned_data_dict)



# Функция для проверки и отображения оставшихся пропусков по каждому активу

def check_missing_data(data_dict, full_names):

    missing_data_summary = []

    for ticker, data in data_dict.items():

        missing_count = data.isnull().sum().sum()  # Суммируем все пропуски по столбцам

        company_name = full_names.get(ticker, "Unknown")  # Получаем полное название компании

        missing_data_summary.append({'Ticker': ticker, 'Company Name': company_name, 'Missing Values': missing_count})



    # Создаем DataFrame для наглядного отображения результатов

    missing_data_df = pd.DataFrame(missing_data_summary)

    display(missing_data_df)



# Проверка на наличие пропусков после заполнения

check_missing_data(filled_data_dict, full_names)

# ============================================
# 6. Первичная визуализация: ресемплирование
# ============================================

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt



# Параметры для отображения графиков

plt.style.use('ggplot')  # Используем стиль ggplot для визуализации



# Функция для визуализации данных с разными интервалами ресемплирования для каждого актива

def plot_resampled_data_individual(filled_data_dict, full_names):

    resample_dict = {

        'D': 'Дни',         # Дни

        'W': 'Недели',      # Недели

        'M': 'Месяцы',      # Месяцы

        'Q': 'Кварталы',    # Кварталы

        '6M': '6 Месяцев',  # 6 месяцев

        'A': 'Годы'         # Годы

    }



    # Создание графиков для каждого актива

    for ticker, data in filled_data_dict.items():

        company_name = full_names.get(ticker, 'Unknown')

        start_date = data.index.min().strftime('%Y-%m-%d')

        end_date = data.index.max().strftime('%Y-%m-%d')



        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))

        fig.suptitle(f'Ресемплированные данные для {company_name} ({ticker})\nПериод: {start_date} - {end_date}', fontsize=16)

        plt.subplots_adjust(hspace=0.4, wspace=0.3)



        for i, (freq, label) in enumerate(resample_dict.items()):

            resampled_data = data['Adj Close'].resample(freq).mean()  # Ресемплирование с расчетом среднего значения



            ax = axes[i // 3, i % 3]

            ax.plot(resampled_data.index, resampled_data, label=f'{company_name} ({ticker}) - {label}')

            ax.set_title(f'{label}')

            ax.set_xlabel('Дата')

            ax.set_ylabel('Цена')

            ax.legend()

            ax.grid(True)



        plt.tight_layout()

        plt.show()



# Вызов функции для отображения графиков для каждого актива

plot_resampled_data_individual(filled_data_dict, full_names)

# ============================================
# 7. Статистические характеристики
# ============================================

In [ ]:
import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt



# Функция для расчета статистических характеристик

def calculate_statistics(data, interval_name, interval_label):

    stats_dict = {

        'Тикер': [],

        'Название': [],

        'Интервал': [],

        'Медиана (абс.)': [],

        'Дисперсия (абс.)': [],

        'Стандартное отклонение (абс.)': [],

        'Волатильность (%)': []

    }

    

    for ticker, df in data.items():

        company_name = full_names.get(ticker, 'Unknown')

        resampled_data = df['Adj Close'].resample(interval_name).mean()

        

        # Заполнение пропусков, чтобы избежать предупреждений

        resampled_data = resampled_data.fillna(method='ffill').fillna(method='bfill')

        

        stats_dict['Тикер'].append(ticker)

        stats_dict['Название'].append(company_name)

        stats_dict['Интервал'].append(interval_label)

        stats_dict['Медиана (абс.)'].append(resampled_data.median())

        stats_dict['Дисперсия (абс.)'].append(resampled_data.var())

        stats_dict['Стандартное отклонение (абс.)'].append(resampled_data.std())

        stats_dict['Волатильность (%)'].append(resampled_data.pct_change().std() * np.sqrt(len(resampled_data)) * 100)

    

    return pd.DataFrame(stats_dict)



# Функция для создания и отображения тепловой карты корреляций

def plot_correlation_heatmap(data, interval_name, interval_label):

    resampled_data = {ticker: df['Adj Close'].resample(interval_name).mean().fillna(method='ffill').fillna(method='bfill') for ticker, df in data.items()}

    correlation_df = pd.DataFrame(resampled_data).corr()

    

    plt.figure(figsize=(10, 8))

    sns.heatmap(correlation_df, annot=False, cmap="coolwarm", cbar=True, xticklabels=correlation_df.columns, yticklabels=correlation_df.index)

    plt.title(f'Корреляционная матрица между активами - {interval_label}')

    

    # Добавляем легенду с полными названиями

    plt.figtext(1.15, 0.5, "\n".join([f"{ticker} - {full_names[ticker]}" for ticker in correlation_df.columns]), horizontalalignment='left', verticalalignment='center', fontsize=10)

    plt.tight_layout()

    plt.show()



# Словарь для интервалов и меток

intervals = {

    'D': 'День',

    'W': 'Неделя',

    'M': 'Месяц',

    'Q': 'Квартал',

    '6M': '6 Месяцев',

    'A': 'Год'

}



# Проходим по каждому временному интервалу, рассчитываем статистику и строим корреляционные матрицы

for interval, label in intervals.items():

    # Рассчёт статистических характеристик

    stats_df = calculate_statistics(filled_data_dict, interval, label)

    print(f"Статистические характеристики - {label}")

    display(stats_df)



    # Построение корреляционной тепловой карты

    plot_correlation_heatmap(filled_data_dict, interval, label)

# ============================================
# 8. Тесты временных рядов (ADF), ADF tests
# ============================================

In [ ]:
import pandas as pd

from statsmodels.tsa.stattools import adfuller



# Функция для выполнения теста ADF по каждому активу и сбор результатов в таблицу

def perform_adf_test(data):

    adf_results = {

        'Тикер': [],

        'Название': [],

        'Test Statistic': [],

        'p-value': [],

        'Critical Value (1%)': [],

        'Critical Value (5%)': [],

        'Critical Value (10%)': [],

        'Стационарность': []

    }

    

    for ticker, df in data.items():

        company_name = full_names.get(ticker, 'Unknown')

        

        # Выполнение теста ADF

        adf_test = adfuller(df['Adj Close'].dropna())

        test_statistic = adf_test[0]

        p_value = adf_test[1]

        critical_values = adf_test[4]

        

        # Определение стационарности

        if p_value < 0.05:

            stationarity = "Стационарен"

        else:

            stationarity = "Нестационарен"

        

        # Добавление данных в таблицу результатов

        adf_results['Тикер'].append(ticker)

        adf_results['Название'].append(company_name)

        adf_results['Test Statistic'].append(test_statistic)

        adf_results['p-value'].append(p_value)

        adf_results['Critical Value (1%)'].append(critical_values['1%'])

        adf_results['Critical Value (5%)'].append(critical_values['5%'])

        adf_results['Critical Value (10%)'].append(critical_values['10%'])

        adf_results['Стационарность'].append(stationarity)

    

    # Преобразование словаря в DataFrame и отображение результатов

    adf_results_df = pd.DataFrame(adf_results)

    return adf_results_df



# Выполнение теста ADF и вывод результатов

adf_results_df = perform_adf_test(filled_data_dict)

print("Результаты теста ADF для всех активов:")

display(adf_results_df)

# ============================================
# 9. Проверка автокорреляции (ACF и PACF)
# ============================================

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.stattools import adfuller

import warnings



# Функция для анализа автокорреляции с ACF и PACF

def autocorrelation_analysis(data, max_lags=20, significance_level=0.05):

    acf_results = {

        'Тикер': [],

        'Название': [],

        'Автокорреляция': []

    }

    

    for ticker, df in data.items():

        company_name = full_names.get(ticker, 'Unknown')

        

        # Проверка на наличие достаточного количества данных для анализа автокорреляции

        if len(df['Adj Close'].dropna()) < max_lags:

            print(f"Недостаточно данных для анализа {ticker} ({company_name}). Пропускаем.")

            acf_results['Тикер'].append(ticker)

            acf_results['Название'].append(company_name)

            acf_results['Автокорреляция'].append("Недостаточно данных")

            continue



        try:

            # Построение ACF и PACF графиков

            fig, axes = plt.subplots(2, 1, figsize=(12, 8))

            plot_acf(df['Adj Close'].dropna(), lags=max_lags, alpha=significance_level, ax=axes[0])

            axes[0].set_title(f'Автокорреляция (ACF) - {company_name} ({ticker})')

            plot_pacf(df['Adj Close'].dropna(), lags=max_lags, alpha=significance_level, ax=axes[1])

            axes[1].set_title(f'Частичная автокорреляция (PACF) - {company_name} ({ticker})')

            plt.tight_layout()

            plt.show()

            

            # Проверка на автокорреляцию

            adf_test = adfuller(df['Adj Close'].dropna())

            p_value = adf_test[1]

            is_autocorrelated = "Имеется автокорреляция" if p_value < significance_level else "Нет автокорреляции"

            

            # Сохранение результатов

            acf_results['Тикер'].append(ticker)

            acf_results['Название'].append(company_name)

            acf_results['Автокорреляция'].append(is_autocorrelated)

        

        except Exception as e:

            print(f"Ошибка при обработке {ticker} ({company_name}): {e}")

            acf_results['Тикер'].append(ticker)

            acf_results['Название'].append(company_name)

            acf_results['Автокорреляция'].append("Ошибка анализа")

    

    # Преобразование результатов в таблицу и отображение

    acf_results_df = pd.DataFrame(acf_results)

    display(acf_results_df)



# Выполнение анализа автокорреляции

autocorrelation_analysis(filled_data_dict, max_lags=20, significance_level=0.05)

# ============================================
# 10. Устранение нестационарности, автокорреляции и сезонности. Возведение в логарифм и перепроверка стацционарности
# ============================================

In [ ]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.stattools import adfuller



# Параметры анализа

window_size = 365  # Rolling window size (in days)

analysis_period = "Период анализа: с {start_date} по {end_date}"



# Получаем даты начала и конца для анализа (предполагаем, что данные заполнены по каждому активу)

start_date = filled_data_dict[list(filled_data_dict.keys())[0]].index.min().strftime('%Y-%m-%d')

end_date = filled_data_dict[list(filled_data_dict.keys())[0]].index.max().strftime('%Y-%m-%d')



# Дефляция данных и вычисление скользящих метрик

log_data_dict = {}

rolling_metrics_dict = {}



for ticker, data in filled_data_dict.items():

    # Логарифмируем данные

    log_data = np.log(data['Adj Close'])

    log_data_dict[ticker] = log_data



    # Вычисляем скользящее среднее и стандартное отклонение

    rolling_mean = log_data.rolling(window=window_size).mean()

    rolling_std = log_data.rolling(window=window_size).std()

    

    rolling_metrics_dict[ticker] = pd.DataFrame({

        'log_price': log_data,

        'rolling_mean': rolling_mean,

        'rolling_std': rolling_std

    })



    # Построение графиков

    plt.figure(figsize=(10, 6))

    plt.plot(log_data, label=f'{full_names[ticker]} ({ticker}) Log Adj Close', color='blue')

    plt.plot(rolling_mean, label='Rolling Mean (365 days)', color='green')

    plt.plot(rolling_std, label='Rolling Std Dev (365 days)', color='red')

    plt.title(f"{full_names[ticker]} ({ticker}) - Log Adj Close, Rolling Mean and Std Dev\n{analysis_period.format(start_date=start_date, end_date=end_date)}")

    plt.xlabel('Date')

    plt.legend()

    plt.show()



# Тест на автокорреляцию и визуализация ACF/PACF

acf_pacf_results = []



for ticker, log_data in log_data_dict.items():

    # ADF тест

    adf_test = adfuller(log_data.dropna())

    adf_result = 'Стационарен' if adf_test[1] < 0.05 else 'Нестационарен'

    

    # Сохраняем результаты теста

    acf_pacf_results.append({

        'Актив': f"{full_names[ticker]} ({ticker})",

        'ADF Test Statistic': adf_test[0],

        'p-value': adf_test[1],

        'Critical Values (1%)': adf_test[4]['1%'],

        'Critical Values (5%)': adf_test[4]['5%'],

        'Critical Values (10%)': adf_test[4]['10%'],

        'Result': adf_result

    })



    # Построение ACF и PACF

    fig, axes = plt.subplots(2, 1, figsize=(10, 8))

    fig.suptitle(f'ACF and PACF for {full_names[ticker]} ({ticker}) - Log Data\n{analysis_period.format(start_date=start_date, end_date=end_date)}')

    plot_acf(log_data.dropna(), ax=axes[0], lags=40, alpha=0.05)

    plot_pacf(log_data.dropna(), ax=axes[1], lags=40, alpha=0.05)

    plt.show()



# Таблица с результатами теста ADF

adf_results_df = pd.DataFrame(acf_pacf_results)

display(adf_results_df)

# Применение дифференцирования к ряду

In [ ]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller, acf, pacf

import seaborn as sns



# Применение дифференцирования

diff_data_dict = {ticker: data.diff().dropna() for ticker, data in log_data_dict.items()}



# Параметры для rolling метрик

rolling_window = 365  # Например, 365 дней



# Построение графиков для каждого актива после дифференцирования

for ticker, diff_data in diff_data_dict.items():

    asset_name = full_names[ticker]

    

    # Скользящее среднее и стандартное отклонение для дифференцированных данных

    roll_mean = diff_data.rolling(window=rolling_window).mean()

    roll_std = diff_data.rolling(window=rolling_window).std()

    

    # График дифференцированных данных и скользящих метрик

    plt.figure(figsize=(14, 7))

    plt.plot(diff_data, color='blue', label=f'{ticker}_log_diff')

    plt.plot(roll_mean, color='green', label=f'roll_mean_log_diff ({rolling_window} дней)')

    plt.plot(roll_std, color='red', label=f'roll_std_log_diff ({rolling_window} дней)')

    plt.title(f'{asset_name} (Дифференцированные логарифмы)')

    plt.legend()

    plt.xlabel('Дата')

    plt.show()

    

    # Тест ADF на стационарность

    adf_result = adfuller(diff_data.dropna())

    print(f"ADF Test для {asset_name} ({ticker}):")

    print(f" - Статистика теста: {adf_result[0]:.4f}")

    print(f" - p-значение: {adf_result[1]:.4f}")

    print(" - Критические значения:")

    for key, value in adf_result[4].items():

        print(f"     {key}: {value:.4f}")

    if adf_result[1] < 0.05:

        print(f"Результат: Временной ряд {asset_name} стационарен.")

    else:

        print(f"Результат: Временной ряд {asset_name} нестационарен.")

    print()



    # Построение графиков автокорреляции и частичной автокорреляции

    fig, axes = plt.subplots(2, 1, figsize=(12, 6))

    fig.suptitle(f'{asset_name} (Автокорреляция и частичная автокорреляция)')



    sns.set_style("darkgrid")

    sns.lineplot(x=range(len(acf(diff_data.dropna(), nlags=40))), y=acf(diff_data.dropna(), nlags=40), ax=axes[0], marker='o')

    sns.lineplot(x=range(len(pacf(diff_data.dropna(), nlags=40))), y=pacf(diff_data.dropna(), nlags=40), ax=axes[1], marker='o')

    

    axes[0].set_title("Autocorrelation")

    axes[1].set_title("Partial Autocorrelation")

    

    plt.tight_layout()

    plt.show()

# ============================================
# 11. Вывод данных после дифференцирования поквартально (необходимо для дальнейшего анализа влияния вместе с макроэкономическими данными)
# ============================================

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt



# A) Поквартальный анализ скорректированных данных для всех активов

def quarterly_analysis(data, ticker, full_name):

    # Применение данных из diff_data_dict (уже дифференцированные)

    quarterly_data = data.resample('Q').mean()



    # Визуализация поквартальных данных

    plt.figure(figsize=(10, 6))

    plt.plot(quarterly_data.index, quarterly_data, label=f'{full_name} ({ticker}) Quarterly Adjusted Close')

    plt.title(f'Поквартальный анализ скорректированных данных для {full_name} ({ticker})')

    plt.xlabel('Дата')

    plt.ylabel('Цена (дифференцированные данные)')

    plt.legend()

    plt.grid(True)

    plt.show()



# Новый словарь для хранения поквартальных данных

quarterly_diff_data_dict = {}



# Применение поквартального анализа для всех активов

for ticker, data in diff_data_dict.items():

    full_name = full_names.get(ticker, ticker)  # Получаем полное название актива

    quarterly_diff_data_dict[ticker] = data.resample('Q').mean()  # Сохраняем поквартальные данные в словарь

    quarterly_analysis(data, ticker, full_name)



# B) Поквартальный анализ по группам активов

def group_quarterly_analysis(group_name, tickers):

    plt.figure(figsize=(12, 8))



    for ticker in tickers:

        data = diff_data_dict[ticker]

        full_name = full_names.get(ticker, ticker)  # Полное название актива

        quarterly_data = data.resample('Q').mean()

        plt.plot(quarterly_data.index, quarterly_data, label=f'{full_name} ({ticker})')



    plt.title(f'{group_name} Group (Quarterly Differenced Data)')

    plt.xlabel('Дата')

    plt.ylabel('Цена (дифференцированные данные)')

    plt.legend()

    plt.grid(True)

    plt.show()



# Применение поквартального анализа для каждой группы активов

for group, tickers in asset_classes.items():

    group_quarterly_analysis(group, tickers)

# ============================================
# 12. Анализ влияния
# ============================================

# Импорт данных с помощью fred api. Macrodata
'''
Этот код загружает и объединяет макроэкономические данные из базы данных FRED (Federal Reserve Economic Data) по следующим показателям: ВВП, уровень безработицы, индекс потребительских цен (CPI), индекс промышленного производства (PMI), процентная ставка, торговый баланс, экспорт и импорт. Данные собираются с 1990 года до текущей даты, после чего объединяются в один `DataFrame` `macro_data` для анализа. В конце отображаются последние строки (с помощью `print(macro_data.tail())`) для предварительного просмотра данных.
'''


In [ ]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss
from scipy import stats
from fredapi import Fred


# Fred API key
fred = Fred(api_key='2409f89109db1e4dcb431b6ca50f575e')

# Устанавливаем временной диапазон
start = datetime.datetime(1990, 1, 1)  # Установим начальную дату как можно раньше
end = datetime.datetime.now()

# Получение данных о ВВП
gdp = fred.get_series('GDP', start, end)

# Получение данных об уровне безработицы
unemployment = fred.get_series('UNRATE', start, end)

# Получение данных об индексе потребительских цен (CPI)
cpi = fred.get_series('CPIAUCSL', start, end)

# Получение данных об индексах промышленного производства (PMI)
pmi = fred.get_series('INDPRO', start, end)

# Получение данных о процентных ставках
interest_rate = fred.get_series('FEDFUNDS', start, end)

# Получение данных о торговом балансе
trade_balance = fred.get_series('BOPGSTB', start, end)

# Получение данных об объемах экспорта и импорта
export_data = fred.get_series('NETEXP', start, end)  # Объем экспорта
import_data = fred.get_series('IMPGS', start, end)  # Объем импорта

# Преобразование данных в DataFrame
macro_data = pd.concat([gdp, unemployment, cpi, pmi, interest_rate, trade_balance, export_data, import_data], axis=1)
macro_data.columns = ['GDP', 'Unemployment Rate', 'CPI', 'PMI', 'Interest Rate', 'Trade Balance', 'Export', 'Import']

# Вывод полученных данных
print(macro_data.tail())

# Ресемплинг поквартально так как на сайте данные даются поквартально 
'''
Код извлекает ключевые макроэкономические показатели (ВВП, безработица, CPI, PMI, процентные ставки, торговый баланс, экспорт, импорт), агрегируя их по кварталам для сглаживания данных. Затем он объединяет эти показатели в один DataFrame для удобного анализа квартальной динамики и сравнения индикаторов.
'''

In [ ]:
# Получение данных о ВВП
gdp = fred.get_series('GDP', start, end).resample('Q').mean()

# Получение данных об уровне безработицы
unemployment = fred.get_series('UNRATE', start, end).resample('Q').mean()

# Получение данных об индексе потребительских цен (CPI)
cpi = fred.get_series('CPIAUCSL', start, end).resample('Q').mean()

# Получение данных об индексах промышленного производства (PMI)
pmi = fred.get_series('INDPRO', start, end).resample('Q').mean()

# Получение данных о процентных ставках
interest_rate = fred.get_series('FEDFUNDS', start, end).resample('Q').mean()

# Получение данных о торговом балансе
trade_balance = fred.get_series('BOPGSTB', start, end).resample('Q').mean()

# Получение данных об объемах экспорта и импорта
export_data = fred.get_series('NETEXP', start, end).resample('Q').mean()  # Объем экспорта
import_data = fred.get_series('IMPGS', start, end).resample('Q').mean()  # Объем импорта

# Преобразование данных в DataFrame
macro_data = pd.concat([gdp, unemployment, cpi, pmi, interest_rate, trade_balance, export_data, import_data], axis=1)
macro_data.columns = ['GDP', 'Unemployment Rate', 'CPI', 'PMI', 'Interest Rate', 'Trade Balance', 'Export', 'Import']

# Вывод полученных данных
print(macro_data.tail())

# ==============================================================
# 13. Описание данных
# ==============================================================

In [ ]:
description = macro_data.describe()
print(description)

# ==============================================================
# 14. Проверка на наличие выбросов
# ==============================================================

'''
Код вычисляет **Z-оценки** для данных в `macro_data`, чтобы определить выбросы: значения, которые более чем на 3 стандартных отклонения отличаются от среднего. Значения `True` в выводе `outliers` указывают на наличие выбросов в соответствующих местах.
'''

In [ ]:
z_scores = np.abs(stats.zscore(macro_data))
outliers = (z_scores > 3)  # Используем Z-Score для определения выбросов
print("Выбросы (значения True указывают на наличие выбросов):")
print(outliers)

# ==============================================================
# 15. Визуализация временных рядов
# ==============================================================

In [ ]:
plt.figure(figsize=(14, 10))
for i, col in enumerate(macro_data.columns):
    plt.subplot(4, 2, i + 1)
    plt.plot(macro_data[col])
    plt.title(col)
    plt.xlabel('Year')
    plt.ylabel(col)
plt.tight_layout()
plt.show()

# Вывод
По графикам выше можно сделать следующие статистические выводы:
1. **GDP (ВВП)**: ВВП показывает стабильный рост с 1990 года, с резкими провалами в периоды экономических кризисов, таких как 2008 и 2020 годы, что указывает на общую тенденцию к росту экономики.
2. **Unemployment Rate (Уровень безработицы)**: Уровень безработицы подвержен циклическим колебаниям, с резкими скачками в периоды экономических кризисов (особенно заметен пик в 2020 году), после чего он постепенно снижается.
3. **CPI (Индекс потребительских цен)**: CPI растет устойчиво на протяжении всего периода, что свидетельствует о долгосрочной инфляции.
4. **PMI (Индекс промышленного производства)**: PMI показывает положительную динамику до 2000-х годов, после чего наблюдаются колебания. Ярко выраженные падения PMI совпадают с экономическими кризисами (например, в 2008 и 2020 годах), указывая на чувствительность промышленного производства к экономическим условиям.
5. **Interest Rate (Процентные ставки)**: Процентные ставки колебались, снижаясь до минимальных уровней в 2010-х годах и снова повышаясь в последние годы, что может быть связано с борьбой с инфляцией и мерами поддержки экономики.
6. **Trade Balance (Торговый баланс)**: Торговый баланс показывает общий нисходящий тренд, что может свидетельствовать о росте дефицита торгового баланса страны.
7. **Export (Экспорт)**: Экспорт показывает тенденцию к снижению после небольшого роста в 1990-х годах, что может указывать на снижение внешней конкурентоспособности.
8. **Import (Импорт)**: Импорт стабильно растет с 1990 года, что может указывать на рост внутреннего потребления и зависимости от иностранных товаров.

В целом, графики показывают долгосрочные тенденции в ключевых экономических показателях, а также влияние глобальных экономических кризисов на безработицу, промышленное производство и процентные ставки.

# ==============================================================
# 16. Тесты на стационарность, автокорреляцию и сезонность
# ==============================================================

In [ ]:
for col in macro_data.columns:
    adf_test = adfuller(macro_data[col].dropna())
    kpss_test = kpss(macro_data[col].dropna(), regression='c')
    
    print(f"\nТесты для {col}:")
    print(f"ADF Statistic: {adf_test[0]}, p-value: {adf_test[1]}")
    print(f"KPSS Statistic: {kpss_test[0]}, p-value: {kpss_test[1]}")

# Вывод
1. **GDP**:
- ADF Statistic: 3.92, p-value: 1.0
- KPSS Statistic: 2.01, p-value: 0.01
- **Вывод**: Невозможно отклонить H0 по ADF, ряд не стационарен. H0 по KPSS отклоняется, что подтверждает нестационарность.

2. **Unemployment Rate**:
- ADF Statistic: -3.14, p-value: 0.024
- KPSS Statistic: 0.18, p-value: 0.1
- **Вывод**: ADF указывает на стационарность (H0 отклоняется), а KPSS не отклоняет H0, указывая на стационарность. Противоречивые результаты, но в целом можно считать ряд стационарным.

3. **CPI**:
- ADF Statistic: 1.90, p-value: 0.999
- KPSS Statistic: 2.05, p-value: 0.01
- **Вывод**: Оба теста показывают, что ряд не стационарен.

4. **PMI**:
- ADF Statistic: -1.98, p-value: 0.295
- KPSS Statistic: 1.64, p-value: 0.01
- **Вывод**: Невозможно отклонить H0 по ADF, ряд не стационарен, а KPSS отклоняет H0, подтверждая нестационарность.

5. **Interest Rate**:
- ADF Statistic: -3.23, p-value: 0.018
- KPSS Statistic: 1.03, p-value: 0.01
- **Вывод**: ADF указывает на стационарность (H0 отклоняется), а KPSS указывает на нестационарность. Можно считать ряд стационарным.

6. **Trade Balance**:
- ADF Statistic: -1.33, p-value: 0.616
- KPSS Statistic: 1.32, p-value: 0.01
- **Вывод**: Невозможно отклонить H0 по ADF, ряд не стационарен, а KPSS подтверждает нестационарность.

7. **Export**:
- ADF Statistic: -1.14, p-value: 0.701
- KPSS Statistic: 1.51, p-value: 0.01
- **Вывод**: Невозможно отклонить H0 по ADF, ряд не стационарен, а KPSS подтверждает нестационарность.

8. **Import**:
- ADF Statistic: -0.15, p-value: 0.944
- KPSS Statistic: 2.03, p-value: 0.01
- **Вывод**: Невозможно отклонить H0 по ADF, ряд не стационарен, а KPSS подтверждает нестационарность.

### Общие выводы:
- Большинство временных рядов (GDP, CPI, PMI, Trade Balance, Export, Import) показывают нестационарность.
- Уровень безработицы и процентная ставка имеют смешанные результаты, но можно считать их стационарными на основании ADF.
- Для анализа данных, возможно, потребуется трансформация (например, дифференцирование) для достижения стационарности, что является необходимым шагом перед дальнейшим моделированием.

# ==============================================================
# 17. Корректировка данных (если необходимо)
# ==============================================================

'''
Разностное преобразование применено для достижения стационарности временных рядов. В результате были получены скорректированные данные, которые позволяют устранить тренды и сезонные колебания, что важно для дальнейшего анализа и моделирования. Результаты разностного преобразования показаны в последней части таблицы скорректированных данных.
'''

In [ ]:
# Разностное преобразование для достижения стационарности
adjusted_macrodata = macro_data.diff().dropna()
print("\nСкорректированные данные (разностное преобразование):")
print(adjusted_macrodata.tail())

# ==============================================================
# 18. Аналитика
# ==============================================================

In [ ]:
adjusted_macrodata.head(14)

# Вывод
Данные по макроданным доступны с 1992-06-30. В связи с этим, с целью анализа влияния и проведения линейной регрессии, мы привели данные по активам и макроданным в единый формат с одинаковым началом временного периода. Помимо синхронизации временных интервалов и ограничения данных общим временным диапазоном, было произведено преобразование данных по активам в общий DataFrame. На основе этого была построена корреляционная матрица в разрезе активов и макроданных.

# Корреляция

In [ ]:
import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

# 1. Синхронизация временных интервалов
# Находим общий диапазон для всех данных

common_start = max(min(adjusted_macrodata.index), min(min(df.index) for df in quarterly_diff_data_dict.values()))
common_end = min(max(adjusted_macrodata.index), max(max(df.index) for df in quarterly_diff_data_dict.values()))

# Ограничиваем данные общим временным диапазоном

macrodata_aligned = adjusted_macrodata.loc[common_start:common_end]
asset_data_aligned = {ticker: df.loc[common_start:common_end] for ticker, df in quarterly_diff_data_dict.items()}

# Преобразование данных по активам в общий DataFrame

combined_asset_data = pd.concat(asset_data_aligned.values(), axis=1)
combined_asset_data.columns = [f"{ticker} ({full_names.get(ticker, ticker)})" for ticker in asset_data_aligned.keys()]

# 2. Приведение к единому формату
# Нормализуем для корректного анализа корреляции

normalized_asset_data = (combined_asset_data - combined_asset_data.mean()) / combined_asset_data.std()
normalized_macrodata = (macrodata_aligned - macrodata_aligned.mean()) / macrodata_aligned.std()

# Соединяем активы и макроданные для анализа корреляции

data_for_correlation = pd.concat([normalized_asset_data, normalized_macrodata], axis=1).dropna()

# 3. Построение корреляционной матрицы

# Вычисляем корреляцию

correlation_matrix = data_for_correlation.corr().loc[normalized_asset_data.columns, normalized_macrodata.columns]

# Визуализация тепловой карты корреляций

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, cmap="coolwarm", cbar_kws={'label': 'Correlation Coefficient'}, square=True, annot=False)
plt.title(f"Корреляционная матрица между активами и макроэкономическими показателями (Период анализа: {common_start.date()} - {common_end.date()})")
plt.xlabel("Макроэкономические показатели")
plt.ylabel("Активы (с полными названиями)")
plt.show()

# 4. Вывод таблицы корреляций для анализа с display
# Таблица корреляций

correlation_table = correlation_matrix.copy()
display(correlation_table)

# Вывод
На основе данной таблицы корреляции между активами и макроэкономическими показателями можно сделать несколько выводов: 
1. **Чувствительность акций к макроэкономическим показателям**: - Акции технологических компаний, таких как Apple (AAPL), Microsoft (MSFT), NVIDIA (NVDA) и Amazon (AMZN), показывают отрицательную корреляцию с процентными ставками. Это значит, что повышение процентных ставок негативно влияет на эти компании, так как более высокие ставки могут повысить затраты на заимствования и снизить их рыночную стоимость. 
2. **Влияние инфляции (ИПЦ) на сырьевые товары**: - Сырьевые товары, такие как нефть (CL=F) и золото (GC=F), имеют положительную корреляцию с индексом потребительских цен (ИПЦ). Это говорит о том, что при росте инфляции цены на сырьевые товары также имеют тенденцию к росту, поскольку они выступают в качестве защиты от инфляции. 
3. **Взаимосвязь валютных курсов и процентных ставок**: - Валютные пары, такие как EURUSD=X (евро/доллар США) и USDJPY=X (доллар США/японская иена), имеют положительную корреляцию с процентными ставками, что подтверждает тенденцию укрепления валют при повышении ставок. Это объясняется тем, что более высокие ставки привлекают иностранные инвестиции, повышая спрос на валюту. 
4. **Зависимость фондовых индексов от экономических условий**: - Основные фондовые индексы, такие как S&P 500 (^GSPC) и Dow Jones (^DJI), имеют положительную корреляцию с ВВП и отрицательную — с уровнем безработицы. Это подтверждает, что рост экономики и снижение безработицы поддерживают фондовые рынки, так как в таких условиях растут корпоративные прибыли и улучшаются ожидания инвесторов. 
5. **Торговый баланс и экспортно-импортные показатели**: - Для таких компаний, как Tesla (TSLA) и Intel (INTC), наблюдается отрицательная корреляция с торговым балансом и положительная корреляция с показателями экспорта. Это говорит о том, что для таких экспортоориентированных компаний увеличение объемов торговли и благоприятные внешнеэкономические условия способствуют их росту. 

Эти выводы позволяют понять, как различные активы реагируют на изменения в экономических показателях и каким образом макроэкономическая ситуация может повлиять на их динамику.

# Линейная регрессия

In [ ]:
import pandas as pd

import statsmodels.api as sm

from IPython.display import display



# 1. Синхронизация временных интервалов

# Определение общего временного диапазона

common_start = max(min(adjusted_macrodata.index), min(min(df.index) for df in quarterly_diff_data_dict.values()))

common_end = min(max(adjusted_macrodata.index), max(max(df.index) for df in quarterly_diff_data_dict.values()))



# Ограничиваем данные общим временным диапазоном

macrodata_aligned = adjusted_macrodata.loc[common_start:common_end]

asset_data_aligned = {ticker: df.loc[common_start:common_end] for ticker, df in quarterly_diff_data_dict.items()}



# Создание таблицы для значимых макроэкономических факторов

significant_factors_summary = pd.DataFrame(columns=['Asset', 'Significant Macroeconomic Factors'])



# 2. Выполнение регрессионного анализа для каждого актива

for ticker, data in asset_data_aligned.items():

    asset_name = full_names.get(ticker, ticker)

    

    # Объединение макроэкономических данных и данных актива

    combined_data = pd.concat([data, macrodata_aligned], axis=1).dropna()

    

    # Макроэкономические показатели в качестве независимых переменных

    X = combined_data[macrodata_aligned.columns]

    X = sm.add_constant(X)  # добавляем константу для регрессии

    y = combined_data['Adj Close']  # зависимая переменная - актив

    

    # Построение регрессионной модели

    model = sm.OLS(y, X).fit()

    results_summary = model.summary2().tables[1]  # извлекаем таблицу коэффициентов

    

    # Вывод таблицы с результатами регрессии

    print(f"\nРегрессионный анализ для {asset_name} ({ticker})")

    print(f"Период анализа: {common_start.date()} - {common_end.date()}")

    display(results_summary)

    

    # 3. Генерация выводов на основе значимых факторов

    significant_factors = results_summary[results_summary['P>|t|'] < 0.05].index.tolist()

    

    # Добавление вывода

    if significant_factors:

        print(f"Значимые макроэкономические факторы для {asset_name} ({ticker}): {', '.join(significant_factors[1:])}")

        # Добавление в сводную таблицу с использованием pd.concat

        significant_factors_summary = pd.concat([

            significant_factors_summary, 

            pd.DataFrame({'Asset': [f"{asset_name} ({ticker})"], 

                          'Significant Macroeconomic Factors': [', '.join(significant_factors[1:])]})

        ], ignore_index=True)

    else:

        print(f"Для {asset_name} ({ticker}) не выявлено значимых макроэкономических факторов на уровне значимости 5%.")



# 4. Отображение сводной таблицы значимых макроэкономических факторов

print("\nСводная таблица значимых макроэкономических факторов для каждого актива:")

display(significant_factors_summary)

# Вывод
Эта таблица отражает результаты регрессионного анализа и показывает значимые макроэкономические факторы, влияющие на различные активы. Каждый актив в списке ассоциируется с одним или несколькими макроэкономическими показателями, которые имеют статистически значимое влияние на его поведение.
1. **Акции крупных компаний**: 
- Для таких компаний, как **Apple (AAPL)** и **NVIDIA (NVDA)**, значимым фактором оказался уровень безработицы. Это указывает на зависимость их деятельности от состояния рынка труда, что может быть связано с потребительскими расходами и спросом. 
- **Meta Platforms (META)** также чувствительна к уровню безработицы, а **Walmart (WMT)** – к уровню безработицы и индексу деловой активности (PMI), что указывает на высокую зависимость от экономической активности и покупательской способности. 
2. **Сырьевые товары**: 
- Цены на **нефть (Crude Oil Futures, CL=F)**, **медь (Copper Futures, HG=F)** и **сою (Soybean Futures, ZS=F)** зависят от показателей импорта, что может свидетельствовать о влиянии мирового спроса и торговых потоков на цены этих товаров. 
- **Золото (Gold Futures, GC=F)** чувствительно к уровню инфляции (CPI), что подтверждает его роль как актива для хеджирования против инфляции. **Серебро (Silver Futures, SI=F)** также коррелирует с CPI, а еще — с торговым балансом и экспортом, что указывает на его зависимость от глобальных экономических факторов. 
3. **Фондовые индексы**: 
- Основные индексы, такие как **S&P 500 (^GSPC)**, **Dow Jones (^DJI)** и **NASDAQ Composite (^IXIC)**, показывают значимую зависимость от уровня безработицы и инфляции (CPI). Это подтверждает их чувствительность к общей экономической ситуации и условиям на рынке труда. 
- **Индекс волатильности (VIX)**, который отражает ожидания инвесторов по колебаниям рынка, также значимо зависит от уровня безработицы и инфляции. 
4. **Гособлигации и ставки**: 
- Для краткосрочных облигаций, таких как **13-недельные казначейские векселя (^IRX)**, значимым фактором является процентная ставка, что логично, учитывая, что их доходность тесно связана с краткосрочными изменениями денежно-кредитной политики. 
- Долгосрочные облигации, такие как **30-летние казначейские облигации (^TYX)**, зависят от уровня безработицы, что указывает на влияние ожиданий по экономическому росту на их доходность. 
5. **Валютные пары**: 
- Для валют, таких как **евро/доллар США (EURUSD=X)** и **японская иена/доллар США (JPYUSD=X)**, значимыми факторами являются экспорт и импорт, что свидетельствует об их чувствительности к торговым потокам. 
- Для **китайского юаня (CNYUSD=X)** и **сингапурского доллара (SGDUSD=X)** значимыми факторами являются торговый баланс, экспорт и импорт, что подтверждает их зависимость от внешнеэкономической торговли и политики. 